# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [88]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [89]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jamestown,42.0970,-79.2353,21.0,68,90,6.91,US,1612829793
1,1,Bagotville,-28.9833,153.4167,78.8,50,53,17.27,AU,1612830030
2,2,Provideniya,64.3833,-173.3000,6.8,78,75,20.13,RU,1612829817
3,3,San Cosme y Damián,-27.3167,-56.3500,86.0,45,20,6.91,PY,1612830033
4,4,Ōdachō-ōda,35.1833,132.5000,35.6,87,75,5.75,JP,1612830034


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [99]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"]

In [103]:
# Plot Heatmap

# figure_layout = {'width': '1000px', 'margin': '0 auto 0 auto'}
# fig = gmaps.figure(layout=figure_layout)
#fig = gmaps.figure(map_type="HYBRID")

fig = gmaps.figure(zoom_level=2, center=(20,20))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
vacay_cities = weather_df[weather_df['Max Temp'] > 70]
vacay_cities = vacay_cities[vacay_cities['Max Temp'] < 80]
vacay_cities = vacay_cities[vacay_cities['Wind Speed'] < 10]
hotel_df = vacay_cities[vacay_cities['Cloudiness'] == 0]

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,Saint-Philippe,-21.3585,55.7679,77.00,61,0,6.91,RE,1612829776
88,88,Río Gallegos,-51.6226,-69.2181,71.60,33,0,8.05,AR,1612830148
127,127,Makkah al Mukarramah,21.4267,39.8261,72.34,35,0,4.47,SA,1612830047
133,133,Kontagora,10.3999,5.4695,72.00,18,0,6.02,NG,1612830212
158,158,Coxim,-18.5067,-54.7600,78.53,56,0,9.22,BR,1612830238
177,177,Salalah,17.0151,54.0924,75.20,46,0,8.05,OM,1612829924
196,196,Camargo,27.6667,-105.1667,74.23,16,0,8.01,MX,1612830298
271,271,Saint-Joseph,-21.3667,55.6167,77.00,61,0,6.91,RE,1612830391
276,276,Santiago,-33.4569,-70.6483,75.20,46,0,8.05,CL,1612830119
358,358,Mildura,-34.2000,142.1500,75.00,43,0,5.75,AU,1612830504


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
hotel_df["Hotel Name"] = ""
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotels = requests.get(base_url, params=params)
    #print(hotels.url)

    # convert to json
    hotels = hotels.json()
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



/Users/lauragabrielson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/lauragabrielson/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [111]:
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,20,Saint-Philippe,-21.3585,55.7679,77.00,61,0,6.91,RE,1612829776,Saint-Philippe
88,88,Río Gallegos,-51.6226,-69.2181,71.60,33,0,8.05,AR,1612830148,Rio Gallegos
127,127,Makkah al Mukarramah,21.4267,39.8261,72.34,35,0,4.47,SA,1612830047,Mecca
133,133,Kontagora,10.3999,5.4695,72.00,18,0,6.02,NG,1612830212,Kontagora
158,158,Coxim,-18.5067,-54.7600,78.53,56,0,9.22,BR,1612830238,Coxim
177,177,Salalah,17.0151,54.0924,75.20,46,0,8.05,OM,1612829924,Salalah
196,196,Camargo,27.6667,-105.1667,74.23,16,0,8.01,MX,1612830298,Camargo
271,271,Saint-Joseph,-21.3667,55.6167,77.00,61,0,6.91,RE,1612830391,Saint-Joseph
276,276,Santiago,-33.4569,-70.6483,75.20,46,0,8.05,CL,1612830119,Santiago
358,358,Mildura,-34.2000,142.1500,75.00,43,0,5.75,AU,1612830504,Mildura


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [114]:
# hotel_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=3,)
locations = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='blue',
    stroke_color='black', scale=2)
    #info_box_content=[f"Hotel: [Hotel Name] for hotel in hotel_info"])


fig = gmaps.figure(zoom_level=21, center=(20,20))
fig.add_layer(hotel_layer)

fig


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [108]:
# Add marker layer ontop of heat map
fig = gmaps.figure(zoom_level=2, center=(20,20))

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layers to the map

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))